In [65]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [66]:
customer = pd.read_csv('Data/customer.csv')
customer

,customer_id,user_id,state,created_at,updated_at,user_nickname
0,23139,12,NORMAL,2019-09-23 23:21,2023-05-01 6:20,iolo
1,23179,18,NORMAL,2019-09-24 2:05,2019-09-24 2:05,smki
2,18948,25,NORMAL,2019-09-09 8:27,2023-01-25 7:17,haed
3,17531,424,NORMAL,2019-09-07 10:10,2023-10-19 1:13,gkaw
4,23028,1103,NORMAL,2019-09-23 8:45,2022-11-21 20:39,kang
...,...,...,...,...,...,...
19679,791375,781335,NORMAL,2022-01-31 22:38,2022-01-31 22:46,dung
19680,791379,781339,NORMAL,2022-01-31 23:12,2023-04-26 7:33,akul
19681,791382,781342,NORMAL,2022-01-31 23:19,2023-04-26 7:33,wwdf
19682,791388,781348,NORMAL,2022-01-31 23:39,2022-11-29 13:35,dusr


In [67]:
customer = customer[(customer['created_at'] <= '2022-01-31')]
customer.reset_index(drop=True, inplace=True)
customer['Event_Name'] = 'Sign_up'
customer.rename(columns={'state' : 'cust_state', 'created_at' : 'Event_Time'}, inplace=True)
customer = customer[['customer_id', 'Event_Name', 'Event_Time']]
customer

,customer_id,Event_Name,Event_Time
0,23139,Sign_up,2019-09-23 23:21
1,23179,Sign_up,2019-09-24 2:05
2,18948,Sign_up,2019-09-09 8:27
3,17531,Sign_up,2019-09-07 10:10
4,23028,Sign_up,2019-09-23 8:45
...,...,...,...
19336,790426,Sign_up,2022-01-30 23:46
19337,790430,Sign_up,2022-01-30 23:49
19338,790431,Sign_up,2022-01-30 23:51
19339,790432,Sign_up,2022-01-30 23:52


In [68]:
order = pd.read_csv('Data/order.csv')

order['discount_price'] = order['discount_price'].fillna(0).astype(int)

order['sale_price'] = order['sale_price'].fillna(order['list_price'] - order['discount_price']).astype(int)

order.rename(columns={'status' : 'ord_status', 'created_at' : 'Event_Time', 'list_price' : 'price', 'discount_price' : 'discount', 'sale_price' : 'Event_Revenue(KRW)'}, inplace=True)

order["Event_Name"] = order["ord_status"].map({
    "CANCELLED": "drop_from_cart",
    "PENDING": "add_to_cart",
    "COMPLETED": "Purchase"
})

order = order[['order_id', 'customer_id', 'course_id', 'Event_Time', 'Event_Name', 'price', 'Event_Revenue(KRW)']]
order

,order_id,customer_id,course_id,Event_Time,Event_Name,price,Event_Revenue(KRW)
0,853323,791388,205080,2022-01-31 23:52,Purchase,156000,136000
1,853322,560400,205080,2022-01-31 23:51,Purchase,156000,156000
2,853319,455290,204013,2022-01-31 23:40,add_to_cart,250000,250000
3,853317,279538,206742,2022-01-31 23:39,Purchase,447000,447000
4,853314,790438,205761,2022-01-31 23:30,Purchase,261000,261000
...,...,...,...,...,...,...,...
27889,773401,288777,205648,2022-01-01 0:23,Purchase,331000,331000
27890,773396,12693,206717,2022-01-01 0:18,Purchase,212000,212000
27891,773394,753996,206331,2022-01-01 0:17,drop_from_cart,240000,240000
27892,773392,12693,206717,2022-01-01 0:13,add_to_cart,212000,212000


In [69]:
course = pd.read_csv('Data/course.csv')

new_categories = ['프로그래밍', '데이터사이언스', '인공지능', '디자인', '영상', '3D', '일러스트', '금융', '마케팅', '업무 생산성', '기획']

def assign_category_new(row):
    if 'B2B' in row['title']:
        return 'B2B'

    elif 'the red :' in row['title'].lower():
        return '더레드'

    elif pd.notna(row['keywords']):
        # 키워드를 소문자로 변환, 띄어쓰기 제거, 중복 제거
        keywords = list(set([keyword.strip().lower().replace(' ', '') for keyword in row['keywords'].split(',')]))
        for category in new_categories:
            formatted_category = category.lower().replace(' ', '')
            if any(formatted_category in keyword for keyword in keywords):
                return category
    else:
        return '기타'

course['category'] = course.apply(assign_category_new, axis=1)

course = course[['course_id', 'title', 'category']]

course

,course_id,title,category
0,211503,뇌 성장과 정서사회,기타
1,209970,(B2B) 와이즈셀,B2B
2,209948,(B2B) 크몽 온,B2B
3,209939,(B2B) K-Di,B2B
4,209938,K-Digital,기타
...,...,...,...
334,201547,구글 애널리틱스 올,마케팅
335,201435,알고리즘 / 기술면,프로그래밍
336,201408,엑셀 실무 마스터,None
337,200399,리더를 위한 HR/,업무 생산성


In [70]:
order = pd.merge(order, course, how='inner', on='course_id')
order

,order_id,customer_id,course_id,Event_Time,Event_Name,price,Event_Revenue(KRW),title,category
0,853323,791388,205080,2022-01-31 23:52,Purchase,156000,136000,나의 칼퇴치트키 엑,업무 생산성
1,853322,560400,205080,2022-01-31 23:51,Purchase,156000,156000,나의 칼퇴치트키 엑,업무 생산성
2,852660,788914,205080,2022-01-31 14:10,Purchase,155000,105000,나의 칼퇴치트키 엑,업무 생산성
3,852315,791047,205080,2022-01-31 13:05,Purchase,155000,105000,나의 칼퇴치트키 엑,업무 생산성
4,852173,531757,205080,2022-01-31 12:17,Purchase,155000,155000,나의 칼퇴치트키 엑,업무 생산성
...,...,...,...,...,...,...,...,...,...
27889,785020,760714,209607,2022-01-05 2:44,add_to_cart,30000,30000,(B2B) K-Di,B2B
27890,784979,753629,209607,2022-01-05 2:26,Purchase,30000,30000,(B2B) K-Di,B2B
27891,784952,760714,209607,2022-01-05 2:22,drop_from_cart,30000,30000,(B2B) K-Di,B2B
27892,785022,761409,209606,2022-01-05 2:44,Purchase,40000,40000,(B2B) K-Di,B2B


In [71]:
refund = pd.read_csv('Data/refund.csv')
refund.dropna(inplace=True)
refund['Event_Name'] = 'Refund'
refund.rename(columns={'created_at' : 'Event_Time', 'amount' : 'Event_Revenue(KRW)'}, inplace=True)
refund = pd.merge(refund, course, how='inner', on='course_id')
refund = refund[['order_id', 'refund_id', 'course_id', 'title', 'category', 'Event_Time', 'Event_Name', 'Event_Revenue(KRW)']]
refund

,order_id,refund_id,course_id,title,category,Event_Time,Event_Name,Event_Revenue(KRW)
0,773436,45470,206185.0,한 번에 끝내는 머,데이터사이언스,2022-01-06 9:46,Refund,-241000
1,777284,45074,206185.0,한 번에 끝내는 머,데이터사이언스,2022-01-04 2:03,Refund,-241000
2,780248,47038,206185.0,한 번에 끝내는 머,데이터사이언스,2022-01-26 5:24,Refund,-160666
3,782746,45703,206185.0,한 번에 끝내는 머,데이터사이언스,2022-01-10 2:45,Refund,-191000
4,785358,45839,206185.0,한 번에 끝내는 머,데이터사이언스,2022-01-10 9:32,Refund,-238108
...,...,...,...,...,...,...,...,...
1003,833215,46938,206023.0,디자인의 차이를 만,디자인,2022-01-25 2:50,Refund,-176000
1004,835343,46914,202712.0,통계와 엑셀을 활용,업무 생산성,2022-01-24 8:02,Refund,-143000
1005,835742,46998,209541.0,7일 완성 패키지,마케팅,2022-01-26 1:52,Refund,-144000
1006,839536,47022,202845.0,GAN 딥러닝 완벽,데이터사이언스,2022-01-26 4:06,Refund,-250000


In [72]:
t = pd.concat([customer, order])
t = pd.concat([t, refund])
t[['customer_id', 'order_id', 'course_id', 'refund_id']] = t[['customer_id', 'order_id', 'course_id', 'refund_id']]
t = t[['customer_id', 'order_id', 'course_id', 'refund_id', 'title', 'category', 'Event_Name', 'Event_Time', 'price', 'Event_Revenue(KRW)']]
t

,customer_id,order_id,course_id,refund_id,title,category,Event_Name,Event_Time,price,Event_Revenue(KRW)
0,23139.0,NaN,NaN,NaN,NaN,NaN,Sign_up,2019-09-23 23:21,NaN,NaN
1,23179.0,NaN,NaN,NaN,NaN,NaN,Sign_up,2019-09-24 2:05,NaN,NaN
2,18948.0,NaN,NaN,NaN,NaN,NaN,Sign_up,2019-09-09 8:27,NaN,NaN
3,17531.0,NaN,NaN,NaN,NaN,NaN,Sign_up,2019-09-07 10:10,NaN,NaN
4,23028.0,NaN,NaN,NaN,NaN,NaN,Sign_up,2019-09-23 8:45,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1003,NaN,833215.0,206023.0,46938.0,디자인의 차이를 만,디자인,Refund,2022-01-25 2:50,NaN,-176000.0
1004,NaN,835343.0,202712.0,46914.0,통계와 엑셀을 활용,업무 생산성,Refund,2022-01-24 8:02,NaN,-143000.0
1005,NaN,835742.0,209541.0,46998.0,7일 완성 패키지,마케팅,Refund,2022-01-26 1:52,NaN,-144000.0
1006,NaN,839536.0,202845.0,47022.0,GAN 딥러닝 완벽,데이터사이언스,Refund,2022-01-26 4:06,NaN,-250000.0


In [73]:
t.to_csv('for_commerce_tableau.csv', index=False)